In [ ]:
from raphtory import Graph
import pandas as pd

# Cargar datos
df = pd.read_csv("kiali_kpi_metrics.csv")

# Crear grafo en Raphtory
g = Graph()

# Crear nodos con propiedades (solo únicos)
nodes = set(df['source_workload'].astype(str)).union(set(df['destination_workload'].astype(str)))
for node in nodes:
    g.add_node(id=str(node), timestamp=0, properties={"service": str(node), "type": "microservice", })

# Crear aristas con propiedades relevantes
for _, row in df.iterrows():
    src = str(row['source_workload'])
    dst = str(row['destination_workload'])
    g.add_edge(src=src, dst=dst, timestamp=int(pd.Timestamp(row.get("timestamp", 0)).timestamp()), properties={
        "status": int(row.get("status", 0)) if "status" in row and not pd.isna(row["status"]) else 0,
        "p90_latency": float(row.get("average_latency", 0)) if not pd.isna(row.get("average_latency", 0)) else 0.0,
        "throughput": float(row.get("throughput", 0)) if not pd.isna(row.get("throughput", 0)) else 0.0,
        "error_rate": float(row.get("error_rate", 0)) if "error_rate" in row and not pd.isna(row["error_rate"]) else 0.0,
        "requests": float(row.get("total_request", 0)) if "total_request" in row and not pd.isna(row["total_request"]) else 0.0
    })

# Verificar estructura del grafo
print(f"Nodos: {len(g.nodes)}")
print(f"Aristas: {len(g.edges)}")

# Exportar a formato JSON para GraphQL
import json
# Custom function to convert the graph to a dictionary format
def graph_to_dict(graph):
    # Extract nodes
    nodes = [{"id": node, "properties": graph.nodes[node]} for node in graph.nodes]
    
    # Extract edges
    edges = []
    for edge in graph.edges:
        src, dst = edge  # Ensure edge is unpacked correctly
        properties = graph.edges[edge]
        edges.append({"src": src, "dst": dst, "properties": properties})
    
    return {"nodes": nodes, "edges": edges}
import shutil  # Importar para eliminar directorios
import os

# Guardar
output_dir = "graphs/final_graph"
if os.path.exists(output_dir):
    shutil.rmtree(output_dir)  # Eliminar el directorio si ya existe
# Guardar grafo
os.makedirs("graphs/", exist_ok=True)
g.save_to_file("graphs/final_graph")
print("✅ Grafo de final guardado.")

Nodos: 11
Aristas: 15
✅ Grafo de final guardado.


In [ ]:
from raphtory.graphql import GraphServer

# Iniciar el servidor GraphQL
server = GraphServer(work_dir="graphs/").start()
client = server.get_client()

# Función segura para ejecutar queries
def safe_query(query, label):
    try:
        result = client.query(query)
        if "graph" in result or "__schema" in result:
            print(f"✅ {label}:")
            return result
        else:
            print(f"⚠️ Advertencia en {label}: Datos devueltos pero no en el formato esperado.")
            return result  # Devolver el resultado para inspección
    except Exception as e:
        print(f"❌ Excepción en {label}: {e}")
        return None

# Query 1: Propiedades de los nodos
node_properties_query = """
{
  graph(path: "final_graph") {
    nodes {
      list {
        name
        degree
        properties {
          keys
          values {
            value
          }
        }
      }
    }
  }
}
"""
node_data = safe_query(node_properties_query, "Node Properties")
if node_data and "graph" in node_data:
    nodes = node_data["graph"]["nodes"]["list"]
    for node in nodes:
        properties = dict(zip(node["properties"]["keys"], [v["value"] for v in node["properties"]["values"]]))
        print(f"Node: {node['name']}, Degree: {node['degree']}, Properties: {properties}")

# Query 2: Propiedades de las aristas
edge_properties_query = """
{
  graph(path: "final_graph") {
    edges {
      list {
        src { name }
        dst { name }
        properties {
          keys
          values { value }
        }
      }
    }
  }
}
"""
edge_data = safe_query(edge_properties_query, "Edge Properties")
if edge_data and "graph" in edge_data:
    edges = edge_data["graph"]["edges"]["list"]
    for edge in edges:
        src = edge["src"]["name"]
        dst = edge["dst"]["name"]
        properties = dict(zip(edge["properties"]["keys"], [v["value"] for v in edge["properties"]["values"]]))
        print(f"Edge: {src} -> {dst}, Properties: {properties}")

# Query 3: Nodos con alto grado
high_degree_nodes_query = """
{
  graph(path: "final_graph") {
    nodes {
      list {
        name
        degree
      }
    }
  }
}
"""
node_degree_data = safe_query(high_degree_nodes_query, "Node Degrees")
if node_degree_data and "graph" in node_degree_data:
    high_degree_nodes = [
        node for node in node_degree_data["graph"]["nodes"]["list"]
        if node["degree"] > 5
    ]
    print("🎯 High Degree Nodes:")
    for node in high_degree_nodes:
        print(f"Node: {node['name']}, Degree: {node['degree']}")

# Query 4: Introspección del esquema
introspection_query = """
{
  __schema {
    types {
      name
      fields {
        name
        type {
          name
          kind
        }
      }
    }
  }
}
"""
schema_info = safe_query(introspection_query, "Schema Introspection")
if schema_info and "__schema" in schema_info:
    types = schema_info["__schema"]["types"]
    for type_info in types:
        print(f"Type: {type_info['name']}")
        if type_info["fields"]:
            for field in type_info["fields"]:
                print(f"  Field: {field['name']}, Type: {field['type']['name']}, Kind: {field['type']['kind']}")

ImportError: cannot import name 'Client' from 'raphtory' (c:\Users\marti\AppData\Local\Programs\Python\Python312\Lib\site-packages\raphtory\__init__.py)